# Benchmark Performance of argmin()

In [16]:
using BenchmarkTools

In [17]:
x = rand(450);

In [18]:
@btime findmin(x)

  556.452 ns (1 allocation: 32 bytes)


(0.004679542062770881, 420)

In [19]:
@btime findmin(@view x[begin:end])

  615.850 ns (3 allocations: 112 bytes)


(0.004679542062770881, 420)

In [20]:
@btime findmin(@view x[1:449])

  581.956 ns (2 allocations: 80 bytes)


(0.004679542062770881, 420)

In [21]:
@btime findmin(@view x[1:448])

  585.171 ns (2 allocations: 80 bytes)


(0.004679542062770881, 420)

In [22]:
@benchmark findmin(x)

BenchmarkTools.Trial: 10000 samples with 186 evaluations.
 Range (min … max):  555.554 ns …  8.464 μs  ┊ GC (min … max): 0.00% … 92.53%
 Time  (median):     558.022 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   573.444 ns ± 83.400 ns  ┊ GC (mean ± σ):  0.14% ±  0.93%

  █▅▄▄▁    ▃▃▃▃▃▁▁▁▁ ▁▁▁▁▁▁▁▁                                  ▁
  █████▇▇▇▆███████████████████████▇█▇████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▆▆▅ █
  556 ns        Histogram: log(frequency) by time       670 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

In [34]:
@code_native findmin(x)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_julia_findmin_2588             ; -- Begin function julia_findmin_2588
	.p2align	2
_julia_findmin_2588:                    ; @julia_findmin_2588
; ┌ @ reducedim.jl:1112 within `findmin`
	.cfi_startproc
; %bb.0:                                ; %top
	stp	x29, x30, [sp, #-16]!           ; 16-byte Folded Spill
	mov	x29, sp
	.cfi_def_cfa w29, 16
; │┌ @ reducedim.jl:1112 within `#findmin#860`
; ││┌ @ reduce.jl:989 within `_findmin`
; │││┌ @ reducedim.jl:1135 within `findmin`
; ││││┌ @ reducedim.jl:1135 within `#findmin#861`
; │││││┌ @ reduce.jl:963 within `_findmin`
; ││││││┌ @ iterators.jl:277 within `pairs` @ iterators.jl:270
; │││││││┌ @ indices.jl:486 within `LinearIndices`
; ││││││││┌ @ abstractarray.jl:98 within `axes`
; │││││││││┌ @ array.jl:149 within `size`
	.cfi_offset w30, -8
	.cfi_offset w29, -16
	ldr	x9, [x0, #8]
; ││││││└└└└
; ││││││┌ @ reduce.jl:170 within `mapfoldl`
; │││││││┌ @ reduce.jl:


; ││││││││││││┌ @ iterators.jl:290 within `_pairs_elt`
; │││││││││││││┌ @ essentials.jl:13 within `getindex`
	mov	x8, sp
	sub	x1, x8, #16
	mov	sp, x1
	stur	x10, [x8, #-16]
Lloh0:
	adrp	x8, _ijl_bounds_error_ints@GOTPAGE
Lloh1:
	ldr	x8, [x8, _ijl_bounds_error_ints@GOTPAGEOFF]
	mov	w2, #1
	blr	x8
LBB0_12:                                ; %L23
; ││││││││││└└└└
; ││││││││││ @ reduce.jl:49 within `foldl_impl`
; ││││││││││┌ @ reduce.jl:378 within `reduce_empty_iter` @ reduce.jl:379
Lloh2:
	adrp	x8, _j_reduce_empty_2590@GOTPAGE
Lloh3:
	ldr	x8, [x8, _j_reduce_empty_2590@GOTPAGEOFF]


	blr	x8
	.loh AdrpLdrGot	Lloh0, Lloh1
	.loh AdrpLdrGot	Lloh2, Lloh3
	.cfi_endproc
; └└└└└└└└└└└
                                        ; -- End function
.subsections_via_symbols


In [24]:
using LoopVectorization

In [25]:
fast_findmin(dij, n) = begin
    best = 1
    @inbounds dij_min = dij[1]
    @turbo for here in 2:n
        newmin = dij[here] < dij_min
        best = newmin ? here : best
        dij_min = newmin ? dij[here] : dij_min
    end
    dij_min, best
end

fast_findmin (generic function with 1 method)

In [26]:
@btime fast_findmin(x, 450)

  99.478 ns (1 allocation: 32 bytes)


(0.004679542062770881, 420)

In [27]:
@benchmark fast_findmin(x, 450)

BenchmarkTools.Trial: 10000 samples with 947 evaluations.
 Range (min … max):   99.482 ns …  1.468 μs  ┊ GC (min … max): 0.00% … 90.26%
 Time  (median):     102.780 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   104.663 ns ± 30.727 ns  ┊ GC (mean ± σ):  0.70% ±  2.22%

  ▇█▆▂▁▁▁▃▆▆▄▄▄▄▂▂▂▂▂▂▃▃▃▂▂▁▁▂▁▂▁▁▁▁▁▁                         ▂
  ████████████████████████████████████████▇▇▇██▇▇▇▇▇▆▆▇▆▆▅▅▅▆▆ █
  99.5 ns       Histogram: log(frequency) by time       122 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

In [28]:
@benchmark for j in 450:-1:1 fast_findmin(x, j) end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  26.792 μs …  1.257 ms  ┊ GC (min … max): 0.00% … 95.82%
 Time  (median):     27.500 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.748 μs ± 20.846 μs  ┊ GC (mean ± σ):  1.22% ±  1.66%

  ▃███▆▆▅▅▅▄▃▂▁▁▁  ▁▁▂▁▁▂▂▂▂▂▂ ▁                              ▂
  ███████████████▇██████████████▇█▇▇▆▆▇▆▇▆▇▇█▇▇▇▆▇▅█▆▆▆▅▅▅▅▅▅ █
  26.8 μs      Histogram: log(frequency) by time        38 μs <

 Memory estimate: 14.06 KiB, allocs estimate: 450.

In [29]:
@benchmark for j in 450:-1:1 findmin(@view x[1:j]) end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  137.208 μs …  2.097 ms  ┊ GC (min … max): 0.00% … 92.12%
 Time  (median):     140.250 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   144.478 μs ± 59.364 μs  ┊ GC (mean ± σ):  1.34% ±  3.04%

  ▄██▇▅▄▃▄▄▅▅▅▄▄▃▃▃▃▃▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁ ▁▁▁                       ▂
  █████████████████████████████████████████████▇█▇█▇▇▇▇▆▆▆▆▇▆▆ █
  137 μs        Histogram: log(frequency) by time       164 μs <

 Memory estimate: 49.22 KiB, allocs estimate: 1350.